In [2]:
import mysql.connector
from mysql.connector import Error


In [11]:
def create_server_connection(host_name, user_name, user_password):

    connection = None

    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            auth_plugin="mysql_native_password",
            buffered=True,
        )

        print("MySQL Database connection successful")
        return connection

    except Error as err:
        print(f"Error: {err}")


pw = "qwerasdf1@"
connection = create_server_connection("localhost", "root", pw)

MySQL Database connection successful


In [4]:
# # Execute sql query
# def execute_query(connection, query):
#     """
#     exqcute a specific query
#     """
#     cursor = connection.cursor()
#     try:
#         cursor.execute(query)
#         connection.commit()
#         print("Query was successful")

#     except Error as err:
#         print(f"Error: '{err}'")


# # execute_query(connection, 'CREATE DATABASE qa_database;')
# # execute_query(connection, 'USE qa_database;')

Query was successful
Query was successful


In [14]:
def execute_query(connection, query):
    """
    return all results from a queries
    """
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        return results

    except Error as err:
        print(f"Error: '{err}'")


query = """


USE qa_database;

"""
execute_query(connection, query)

Error: 'No result set to fetch from.'


In [15]:
query = """

CREATE TABLE qa_table2 (
    question TEXT,
    answer TEXT
);

"""
execute_query(connection, query)

Error: '1050 (42S01): Table 'qa_table2' already exists'


In [16]:
# Insert some common questions and answers into the database
insert_data_query = """

INSERT INTO qa_table (question, answer) VALUES 
(
    ("What is your name?", "My name is Basel."),
    ("What is the capital of France?", "PSG"),
    ("How old are you?", "I am an AI bot.")
);

"""

select_query = """SELECT * FROM qa_table;"""

execute_query(connection, select_query)

[('What is your name?', 'My name is Basel'),
 ('What is the capital of France?', 'Paris'),
 ('How old are you?', 'I am an AI bot'),
 ('what is education?',
  ' offers advanced instruction in specific fields of study.'),
 ('what is education??',
  ' offers advanced instruction in specific fields of study.')]

In [18]:
question = "What is your name?"
db_cursor = connection.cursor()
db_cursor.execute("SELECT answer FROM qa_table WHERE question = %s", (question,))
result = db_cursor.fetchone()
result[0]


'My name is Basel'

In [43]:
import difflib 
all_questions = ["one","two","three","four","five"]
most_similar_question = difflib.get_close_matches("fivvve", all_questions, n=1)    
most_similar_question

['five']

In [45]:
difflib.SequenceMatcher(None, "fivvve", most_similar_question[0]).ratio()

0.8